### Solve IK for `Common Rig`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r,r2quat
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig.xml`

In [4]:
xml_path = '../asset/myo_sim/human/myohuman_v0.1(mj236).mjb'
env = MuJoCoParserClass(name='MyoSuite',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

ValueError: mj_loadModel: failed to load from mjb

### Loop

In [7]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='MyoSuite',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])
env.reset()

# Set which joints to control
rev_joint_names = env.rev_joint_names
joint_idxs_fwd = [env.model.joint(jname).qposadr[0] for jname in rev_joint_names]
joint_idxs_jac  = [env.model.joint(jname).dofadr[0] for jname in rev_joint_names]
q_rev_init = env.get_qpos_joints(rev_joint_names)
n_rev_joint = len(rev_joint_names)

# Set IK targets
ik_body_names,ik_p_trgts,ik_R_trgts,IK_Ps,IK_Rs = [],[],[],[],[]
# IK target 1
ik_body_name = 'distph3'
p_curr,R_curr = env.get_pR_body(body_name=ik_body_name)
ik_p_trgt = p_curr+np.array([+0.3,-0.1,-0.0])
ik_R_trgt = rpy2r(np.radians([0,-30,30]))@R_curr
IK_P = True
IK_R = True
ik_body_names.append(ik_body_name)
ik_p_trgts.append(ik_p_trgt)
ik_R_trgts.append(ik_R_trgt)
IK_Ps.append(IK_P)
IK_Rs.append(IK_R)
# IK target 2
ik_body_name = '3distph_l'
p_curr,R_curr = env.get_pR_body(body_name=ik_body_name)
ik_p_trgt = p_curr+np.array([-0.3,+0.1,0.0])
ik_R_trgt = rpy2r(np.radians([0,30,-30]))@R_curr
IK_P = True
IK_R = True
ik_body_names.append(ik_body_name)
ik_p_trgts.append(ik_p_trgt)
ik_R_trgts.append(ik_R_trgt)
IK_Ps.append(IK_P)
IK_Rs.append(IK_R)

# Reset and loop 
q = q_rev_init.copy()
FIRST_FLAG = True
while env.is_viewer_alive():
    
    # Solve inverse kinematics
    J_list,ik_err_list = [],[]
    for ik_idx,ik_body_name in enumerate(ik_body_names):
        ik_p_trgt = ik_p_trgts[ik_idx]
        ik_R_trgt = ik_R_trgts[ik_idx]
        IK_P = IK_Ps[ik_idx]
        IK_R = IK_Rs[ik_idx]
        J,ik_err = env.get_ik_ingredients(
            body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
            IK_P=IK_P,IK_R=IK_R)
        J[:,:6] = 0.0 # ignore base joints
        J_list.append(J)
        ik_err_list.append(ik_err)
    J_stack      = np.vstack(J_list)
    ik_err_stack = np.hstack(ik_err_list)
    dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
    q = q + dq[joint_idxs_jac]
    
    # FK
    PERTURB_BASE_POSITION = True
    if PERTURB_BASE_POSITION:
        jntadr = env.model.body('body_root').jntadr[0]
        qposadr = env.model.jnt_qposadr[jntadr]
        base_pos = env.data.qpos[qposadr:qposadr+3]
        base_pos = base_pos + 0.0*np.random.randn(3) # modify base position
        env.data.qpos[qposadr:qposadr+3] = base_pos
        env.data.qpos[qposadr+3:qposadr+7] = r2quat(rpy2r(np.radians([0,0,0])))
    env.forward(q=q,joint_idxs=joint_idxs_fwd,INCREASE_TICK=True)
    
    if env.loop_every(tick_every=10):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        
        # Text information
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3),PLOT_AXIS=False,
                   label='Tick:[%d]'%(env.tick))
        
        # Plot bodies
        env.plot_body_T(body_name='body_root',PLOT_AXIS=True,axis_len=0.5,axis_width=0.01)
        # for body_name in env.body_names:
        #     p,R = env.get_pR_body(body_name=body_name)
        #     env.plot_T(p=p,R=R,PLOT_AXIS=True,axis_len=0.05,axis_width=0.005)
            
        # Plot IK target
        for ik_idx,ik_body_name in enumerate(ik_body_names):
            ik_p_trgt = ik_p_trgts[ik_idx]
            ik_R_trgt = ik_R_trgts[ik_idx]
            env.plot_body_T(body_name=ik_body_name,
                            PLOT_AXIS=True,axis_len=0.2,axis_width=0.01)
            env.plot_T(p=ik_p_trgt,R=ik_R_trgt,
                       PLOT_AXIS=True,axis_len=0.2,axis_width=0.01)
        
        # Plot revolute joints with arrow
        # for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
        #     axis_joint = env.model.jnt_axis[rev_joint_idx]
        #     p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
        #     axis_world = R_joint@axis_joint
        #     axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.2)
        #     axis_len,axis_r = 0.1,0.01
        #     env.plot_arrow_fr2to(
        #         p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
        #         r=axis_r,rgba=axis_rgba)
            
        # Plot contact information
        # env.plot_contact_info(h_arrow=0.3,rgba_arrow=[1,0,0,1],
        #                       PRINT_CONTACT_BODY=False)
        env.render()
        
    # Save image
    if env.is_viewer_alive() and FIRST_FLAG:
        scene_img_init = env.grab_image()
    if env.is_viewer_alive() and env.loop_every(tick_every=100):
        scene_img_final = env.grab_image()
    FIRST_FLAG = False
        
# Close MuJoCo viewer
env.close_viewer()

# Plot initial and final scenes
fig,axs = plt.subplots(1,2,figsize=(10,4))
plt.subplots_adjust(top=0.85,bottom=0.15,wspace=0.2,hspace=0.5)
axs[0].imshow(scene_img_init)
axs[0].set_title('Initial Scene',fontsize=11)
axs[1].imshow(scene_img_final)
axs[1].set_title('Final Scene (tick:[%d])'%(env.tick),fontsize=11)
plt.show()

KeyError: "Invalid name 'distph3'. Valid names: ['calcn_l', 'calcn_r', 'femur_l', 'femur_r', 'head', 'patella_l', 'patella_r', 'pelvis', 'root', 'talus_l', 'talus_r', 'tibia_l', 'tibia_r', 'toes_l', 'toes_r', 'torso', 'world']"

In [ ]:
env.open_interactive_viewer()